In [14]:
import numpy as np
import pandas as pd
import itertools


[data source](https://www.census.gov/programs-surveys/acs/data/pums.html)

In [62]:
#data sample as numpy array, first three rows of 2015 New York PUMS
with open("ss15hny.csv") as t_in:
    np_data = np.genfromtxt(itertools.islice(t_in, 3), names=True, delimiter=',', dtype=object)

In [80]:
#entire csv as panda dataframe
df = pd.read_csv("ss15hny.csv",header=0)
# question... what is default missing data representation
# what does later summary functions (mean) do with missing cells?

In [20]:
#column headers
print np_data.dtype.names


('RT', 'SERIALNO', 'DIVISION', 'PUMA', 'REGION', 'ST', 'ADJHSG', 'ADJINC', 'WGTP', 'NP', 'TYPE', 'ACCESS', 'ACR', 'AGS', 'BATH', 'BDSP', 'BLD', 'BROADBND', 'BUS', 'COMPOTHX', 'CONP', 'DIALUP', 'DSL', 'ELEP', 'FIBEROP', 'FS', 'FULP', 'GASP', 'HANDHELD', 'HFL', 'INSP', 'LAPTOP', 'MHP', 'MODEM', 'MRGI', 'MRGP', 'MRGT', 'MRGX', 'OTHSVCEX', 'REFR', 'RMSP', 'RNTM', 'RNTP', 'RWAT', 'RWATPR', 'SATELLITE', 'SINK', 'SMP', 'STOV', 'TEL', 'TEN', 'TOIL', 'VACS', 'VALP', 'VEH', 'WATP', 'YBL', 'FES', 'FFINCP', 'FGRNTP', 'FHINCP', 'FINCP', 'FPARC', 'FPLMPRP', 'FSMOCP', 'GRNTP', 'GRPIP', 'HHL', 'HHT', 'HINCP', 'HOTWAT', 'HUGCL', 'HUPAC', 'HUPAOC', 'HUPARC', 'KIT', 'LNGI', 'MULTG', 'MV', 'NOC', 'NPF', 'NPP', 'NR', 'NRC', 'OCPIP', 'PARTNER', 'PLM', 'PLMPRP', 'PSF', 'R18', 'R60', 'R65', 'RESMODE', 'SMOCP', 'SMX', 'SRNT', 'SSMC', 'SVAL', 'TAXP', 'WIF', 'WKEXREL', 'WORKSTAT', 'FACCESSP', 'FACRP', 'FAGSP', 'FBATHP', 'FBDSP', 'FBLDP', 'FBROADBNDP', 'FBUSP', 'FCOMPOTHXP', 'FCONP', 'FDIALUPP', 'FDSLP', 'FELEP',


What are these columns? 
[Codes for column headers and cell data](https://www2.census.gov/programs-surveys/acs/tech_docs/pums/data_dict/PUMSDataDict15.pdf)

Each row is a household or family. Which metrics do we want on the map and which data could compose them?
 
* Income
    + FINCP (family income)
    + ADJINC (Use ADJINC to adjust FINCP to constant dollars. But it seems to be the same for all data in this sample.)
* Rent burden
    + GRNTP (gross rent)
* Race
    + RAC1P (race, i.e. Black White American Indian)
    + ANC1P (ancestry, i.e. Mexican, Mexican American, or Hispanic)
* Location
    + PUMA (Public use microdata area code) 
    + ST (state)

[Guide to PUMS](https://www.census.gov/content/dam/Census/library/publications/2009/acs/ACSPUMS.pdf)

[More documentation](https://www2.census.gov/programs-surveys/acs/tech_docs/pums/ACS2015_PUMS_README.pdf)

[PUMA shapefiles download link from NYU](https://geo.nyu.edu/catalog/nyu_2451_34562)

[My Carto project](https://aprilrabkin.carto.com/builder/b10e6e90-926b-11e7-a56e-0ee462b5436c)

[Pandas cheatsheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)

In [21]:
print df.shape #rows and columns

(92817, 235)


In [63]:
print df[['PUMA','FINCP','GRNTP']].head(10)

   PUMA     FINCP   GRNTP
0  3806       NaN  2760.0
1  3205   26020.0     NaN
2  3312       NaN     NaN
3  4004   15600.0   420.0
4  1500       NaN     NaN
5  1400       NaN     NaN
6  1500   81800.0     NaN
7   904       NaN     NaN
8  3203   46040.0     NaN
9   200  102600.0     NaN


In [51]:
print df.shape

(92817, 235)


In [59]:
print df.PUMA.nunique()
print df.FINCP.nunique()
print df.GRNTP.nunique()
#print df.PUMA.value_counts()

145
7873
1685


In [64]:
print df.ST.value_counts() #Make sure it's only NY State data

36    92817
Name: ST, dtype: int64


In [90]:
#filter out 232 other columns
df1 = df[['PUMA','FINCP','GRNTP']]

In [81]:
#what if we filter out records with null values in any of these three columns?
print df[pd.notnull(df['FINCP'])].shape
print df[pd.notnull(df['GRNTP'])].shape 

(48436, 235)
(26560, 235)


In [83]:
# only 48,436 reporting income
# only 26,560 reporting rent
# next: build table of average GRNTP * 12 / average FINCP for each PUMA

In [129]:
avgs_per_puma = df1.groupby(by='PUMA').mean() #does this ignore the NaNs?

In [130]:
#print avgs_per_puma
print type(avgs_per_puma)

<class 'pandas.core.frame.DataFrame'>


In [142]:
avgs_per_puma['rent_burden'] = avgs_per_puma['GRNTP']*12/avgs_per_puma['FINCP']

In [143]:

print avgs_per_puma.columns

Index([u'FINCP', u'GRNTP', u'rent_burden', u'rent_burden_percent'], dtype='object')


In [145]:
avgs_per_puma.to_csv('rent_burden.csv')

In [144]:
print avgs_per_puma

              FINCP        GRNTP  rent_burden  rent_burden_percent
PUMA                                                              
100    68960.468271   836.325581     0.145531                   14
200    73745.153614   811.250000     0.132009                   13
300    95146.198630   859.746377     0.108433                   10
401    78560.884058   698.226415     0.106653                   10
402    81234.534826   781.323276     0.115417                   11
403    79536.617886   834.704819     0.125935                   12
500    76505.660194   920.662338     0.144407                   14
600    70220.836406   730.755556     0.124878                   12
701    68064.697368   828.530055     0.146072                   14
702    97361.892351   946.548077     0.116663                   11
703   106445.956967   976.235294     0.110054                   11
704    92479.949627   727.507576     0.094400                    9
800    83291.344196   747.440299     0.107686                 

In [146]:
# having trouble joining this table with the puma shapefile in carto's sql console, so doing it here
import geopandas as gpd
shapefile_df = gpd.read_file('shapefiles/nyu_2451_34562.shp')
print shapefile_df.head

<bound method GeoDataFrame.head of     PUMA    Shape_Area     Shape_Leng  \
0   3701  9.793278e+07   53287.177054   
1   3702  1.889968e+08  106047.764767   
2   3704  1.062121e+08   47971.694985   
3   3705  1.224932e+08   68592.769561   
4   3706  4.388333e+07   52089.611906   
5   3707  4.227752e+07   37580.013431   
6   3708  5.589653e+07   34853.551479   
7   3709  1.241064e+08   73155.156201   
8   3710  1.377953e+08   91133.335817   
9   3801  8.125211e+07   64047.071933   
10  3802  4.689992e+07   37923.269524   
11  3803  3.983944e+07   38193.793288   
12  3804  6.462190e+07   63165.855753   
13  3805  5.516227e+07   52971.675191   
14  3806  8.849629e+07   47541.367648   
15  3807  8.560316e+07   72184.063758   
16  3808  4.496904e+07   42380.091372   
17  3809  4.869627e+07   35283.884764   
18  3810  8.092287e+07  122306.680573   
19  3901  5.715762e+08  194788.223429   
20  3902  6.760010e+08  153438.155956   
21  3903  3.762426e+08  158125.451155   
22  4001  1.149370e+08

In [194]:
print avgs_per_puma['PUMA'].dtype
print shapefile_df['PUMA'].dtype
print shapefile_df['PUMA'].apply(np.int64)
shapefile_df['PUMA'] = shapefile_df['PUMA'].apply(np.int64)
print shapefile_df['PUMA'].dtype

int64
object
0     3701
1     3702
2     3704
3     3705
4     3706
5     3707
6     3708
7     3709
8     3710
9     3801
10    3802
11    3803
12    3804
13    3805
14    3806
15    3807
16    3808
17    3809
18    3810
19    3901
20    3902
21    3903
22    4001
23    4002
24    4003
25    4004
26    4005
27    4006
28    4007
29    4008
30    4009
31    4010
32    4011
33    4012
34    4013
35    4014
36    4015
37    4016
38    4017
39    4018
40    4101
41    4102
42    4103
43    4104
44    4105
45    4106
46    4107
47    4108
48    4109
49    4110
50    4111
51    4112
52    4113
53    4114
54    3703
Name: PUMA, dtype: int64
int64


In [159]:
avgs_per_puma['PUMA'] = avgs_per_puma.index

In [198]:
#print shapefile_df.iloc[0].geometry
#print avgs_per_puma['PUMA']
#print shapefile_df.sort_values(by='PUMA',ascending=False)
#print avgs_per_puma.sort_values(by='PUMA',ascending=False)

In [216]:
#print pd.merge(avgs_per_puma, shapefile_df, how='right', on='PUMA').head
#print shapefile_df[shapefile_df.PUMA.isin(avgs_per_puma.PUMA)].head
#print avgs_per_puma.PUMA.head(3)
#print shapefile_df.PUMA.head(3)
joined = pd.merge(shapefile_df, avgs_per_puma, how='left', on='PUMA', left_index=True)


In [217]:
print joined.columns

Index([               u'PUMA',          u'Shape_Area',          u'Shape_Leng',
                  u'geometry',               u'FINCP',               u'GRNTP',
               u'rent_burden', u'rent_burden_percent'],
      dtype='object')


In [219]:
joined = joined.drop(['rent_burden'],axis=1)

In [220]:
print joined.columns

Index([               u'PUMA',          u'Shape_Area',          u'Shape_Leng',
                  u'geometry',               u'FINCP',               u'GRNTP',
       u'rent_burden_percent'],
      dtype='object')


In [221]:
print joined.shape

(55, 7)


In [222]:
joined.to_csv('joined.csv')

[how to georeference on carto](https://carto.com/learn/guides/analysis/georeference)

In [228]:
shapefile_df.geometry

0     POLYGON ((1006076.28338623 262138.1090087891, ...
1     POLYGON ((1021632.335632324 267934.4393920898,...
2     POLYGON ((1026308.769592285 256767.6972045898,...
3     POLYGON ((1020434.516601562 248704.9116210938,...
4     POLYGON ((1011917.66619873 255535.5838012695, ...
5     POLYGON ((1012009.464599609 253420.0203857422,...
6     POLYGON ((1005060.892822266 247052.5200195312,...
7     (POLYGON ((1029456.000793457 237188.9177856445...
8     (POLYGON ((1012821.805786133 229228.2645874023...
9     (POLYGON ((1004601.953430176 259027.5151977539...
10    POLYGON ((1001062.708984375 241053.7687988281,...
11    POLYGON ((1001142.377197266 241305.2088012695,...
12    (POLYGON ((1006028.597595215 231058.7962036133...
13    (POLYGON ((1000370.938598633 219466.6605834961...
14    POLYGON ((989355.5845947266 219020.2369995117,...
15    POLYGON ((985170.3721923828 221087.3887939453,...
16    (POLYGON ((994681.4056396484 203127.6748046875...
17    (POLYGON ((989137.1102294922 196325.438781